In [1]:
import os
import sys
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent.parent) +'/')
from utils.visualize import img_grid

from torchvision import transforms 

import numpy as np
import pandas as pd
import torchio as tio
import nibabel as nib

import ipywidgets as ipyw
import matplotlib.pyplot as plt
import SimpleITK as sitk
import torch
from torch import nn
from  torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from torchmetrics.image.fid import FrechetInceptionDistance
import scipy
from sklearn import metrics
import json 

In [2]:
Path(os.getcwd()).parent.parent

PosixPath('/home/miplab/Projects/macaw')

In [3]:
macaw_path = str(Path(os.getcwd()).parent.parent)
ukbb_path = '/home/miplab/ukbb'
ukbb_T1_warped = ukbb_path + '/T1_warped'
generated_path = ukbb_path + '/generated_images/PCA'
folder_bids_original = '/home/miplab/ukbb/PCA_Bids_original_2'
filtered_path = generated_path + '/median_filtered'
diffs_folder =  ukbb_path + '/generated_images/PCA_diffs'
mriqc_generated = ukbb_path + '/generated_images/MRIQC_post_2'
mriqc_original = ukbb_path + '/MRIQC_original_2'
z_initial = 41
z_fim = 140
nsamples = 5

In [4]:
data_path = ukbb_path + '/ukbb_img.csv'
df = pd.read_csv(data_path,low_memory=False)

In [5]:
df

,eid,Sex,Age,BMI,Stem_4th_Ven
0,1020226,0.0,55.0,22.3386,18723.0
1,1020261,1.0,60.0,27.2576,28261.0
2,1020278,1.0,67.0,35.8180,23954.0
3,1020394,1.0,52.0,35.6728,29364.0
4,1020402,1.0,60.0,27.0801,21178.0
...,...,...,...,...,...
40191,6023604,0.0,62.0,20.7491,23976.0
40192,6023739,0.0,52.0,20.5239,20992.0
40193,6023849,1.0,66.0,28.1214,28194.0
40194,6023960,0.0,62.0,28.2426,21587.0


In [6]:
df['Age'].describe()

count    40196.000000
mean        63.938949
std          7.690955
min         44.000000
25%         58.000000
50%         64.000000
75%         70.000000
max         82.000000
Name: Age, dtype: float64

In [7]:
# Getting the generated images
subjects_eid = np.array([])
cf_age = np.array([])
cf_sex = np.array([])
images_generated = []
for file in tqdm(os.listdir(generated_path)):
    if '.nii' in file:
        splits = file.split("_")
        subject_eid = int(splits[0])
        age = splits[1]
        sex = splits[2].split(".")[0]
        subjects_eid = np.append(subjects_eid,subject_eid)
        cf_age = np.append(cf_age, age)
        cf_sex = np.append(cf_sex, sex)
        image_load = nib.load(os.path.join(generated_path, file)).get_fdata()
        images_generated.append(image_load)
print(len(images_generated))
images_generated = np.array(images_generated)

  0%|          | 0/200 [00:00<?, ?it/s]

200


In [8]:
# Getting the original images
images_original = []
real_age = []
real_sex = []
for individual in tqdm(subjects_eid):
    individual = int(individual)
    real_age.append(df[df["eid"] == individual]["Age"].item())
    real_sex.append(df[df["eid"] == individual]["Sex"].item())
    image_load = nib.load(os.path.join(ukbb_T1_warped, str(individual) + ".nii.gz")).get_fdata()
    images_original.append(image_load)
real_age = np.array(real_age)
real_sex = np.array(real_sex)
images_original = np.array(images_original)

  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
images_original = np.array(images_original)
type(images_original)

In [ ]:
#cropping original image
generated_shape = images_generated[0].shape
original_shape = images_original[0].shape
x_initial = int((original_shape[0] - generated_shape[0])/2)
x_fim = x_initial +  generated_shape[0]
y_initial = int((original_shape[1] - generated_shape[1])/2)
y_fim = y_initial +  generated_shape[1]
images_original = [image[x_initial:x_fim,y_initial:y_fim,z_initial:z_fim+1] for image in images_original]

In [ ]:
images_original_normalized = np.copy(images_original)

In [ ]:
# Normalizing
max_min_original = []
for one_image in images_original_normalized:
    slices = one_image.shape[2]
    max_min_slice = []
    for z in range(slices):
        maxv = np.max(one_image[:,:,z])
        minv = np.min(one_image[:,:,z])
        one_image[:,:,z] = ((one_image[:,:,z] - minv) / maxv)
        max_min_slice.append({
            'min': minv,
            'max': maxv,
        })
    max_min_original.append(max_min_slice)

In [ ]:
type(images_original_normalized)

## Difference map

In [ ]:
rands = np.random.randint(0,len(images_generated),nsamples) 
images_generated_rand = images_generated[rands]
image_original_rand = images_original_normalized[rands]
subjects_eid_rand  = subjects_eid[rands].astype(int)
cf_age_rand  = cf_age[rands]
cf_sex_rand  = cf_sex[rands]
real_age_rand = real_age[rands]
real_sex_rand = real_sex[rands]

In [ ]:
subjects_eid[rands]

In [ ]:
titles_cf = [f'Age:{a}, Sex:{s}' for a,s in zip(cf_age_rand,cf_sex_rand)]
real_sex_rand = ['M' if round(s) else 'F' for s in real_sex_rand]
titles_real = [f'Age:{a}, Sex:{s}' for a,s in zip(real_age_rand,real_sex_rand)]

In [ ]:
# Mapping the difference

diff = [np.subtract(a,s) for a,s in zip(image_original_rand,images_generated_rand)]

In [ ]:
def plot_slice(z_slice):
    order_slice = z_slice - z_initial
    plt.rcParams["figure.figsize"] = (20,5)
    img_grid([one_image[:,:,order_slice] for one_image in image_original_rand],cols=nsamples,titles=titles_real)
    img_grid([one_image[:,:,order_slice] for one_image in images_generated_rand],cols=nsamples,titles=titles_cf)
    img_grid([one_image[:,:,order_slice] for one_image in diff],cols=nsamples, cmap='seismic', clim=(-1,1))

In [ ]:
ipyw.interact(plot_slice, 
              z_slice=ipyw.IntSlider(min=z_initial, max=z_fim, step=1, continuous_update=False, description='Image Slice:'))

## Saving original cropped and Diff in files

In [ ]:
images_original[0].shape

In [ ]:
#saving the original just cropped
for index in range(len(subjects_eid)):
    eid = str(int(subjects_eid[index]))
    folder = 'sub-' + eid
    new_name = f"sub-{eid}_ses-01_acq-ShortExample_run-01_T1w.nii.gz"
    new_path = folder_bids_original + "/" + folder + '/ses-01/anat'
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    one_original = nib.load(os.path.join(ukbb_T1_warped, eid + ".nii.gz"))
    # Saving new original images
    ni_img = nib.Nifti1Image(images_original[index], one_original.affine, one_original.header)
    nib.save(ni_img,new_path + "/" + new_name)
    #shutil.copyfile(project + "/" + prefixed[0], new_path + "/" + new_name)

In [ ]:
#saving the original cropped and normalized
for index in range(len(subjects_eid)):
    eid = str(int(subjects_eid[index]))
    folder = 'sub-' + eid
    new_name = f"sub-{eid}_ses-01_acq-ShortExample_run-01_T1w.nii.gz"
    new_path = folder_bids_original + "/" + folder + '/ses-01/anat'
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    one_original = nib.load(os.path.join(ukbb_T1_warped, eid + ".nii.gz"))
    # Saving new original images
    ni_img = nib.Nifti1Image(images_original_normalized[index], one_original.affine, one_original.header)
    nib.save(ni_img,new_path + "/" + new_name)
    #shutil.copyfile(project + "/" + prefixed[0], new_path + "/" + new_name)

In [ ]:
# getting all diffs
all_diff = [np.subtract(a,s) for a,s in zip(images_original_normalized,images_generated)]

In [ ]:
#saving the diff

for index in range(len(subjects_eid)):
    eid = str(int(subjects_eid[index]))
    one_original = nib.load(os.path.join(ukbb_T1_warped, eid + ".nii.gz"))
    ni_img = nib.Nifti1Image(all_diff[index], one_original.affine, one_original.header)
    nib.save(ni_img,diffs_folder + f"/{eid}.nii.gz")

## Frechet Inception Distance (FID) Inceptionv3

In [ ]:
torch_images_original = torch.from_numpy(images_original_normalized)
torch_images_generated = torch.from_numpy(images_generated)
# Add channel dimension
torch_images_original = torch_images_original[:,None, :,:, :]
torch_images_generated = torch_images_generated[:,None, :,:, :]

In [ ]:
# https://torchmetrics.readthedocs.io/en/stable/image/frechet_inception_distance.html

batch_size, channel, x_size, y_size, z_size = torch_images_original.shape

torch_images_original = nn.functional.interpolate(torch_images_original, size=[299, 299,z_size])
torch_images_generated = nn.functional.interpolate(torch_images_generated,size=[299, 299,z_size])

In [ ]:
padding = torch.zeros(batch_size, 2, 299, 299, z_size)

torch_images_original = torch.cat((torch_images_original, padding), 1)
torch_images_generated = torch.cat((torch_images_generated, padding), 1)

In [ ]:
metric = FrechetInceptionDistance(feature=2048, normalize=True)
np_fid = np.array([])

for z_slice in tqdm(range(np_images_generated.shape[3])):
    metric.update(torch_images_original[:,:,:,:,z_slice], real=True)
    metric.update(torch_images_generated[:,:,:,:,z_slice], real=False)
    np_fid = np.append(np_fid, metric.compute())
    metric.reset()

In [ ]:
np_fid.mean()

In [ ]:
np_fid.std()

In [ ]:
np_fid.min()

In [ ]:
np_fid.max()

## IQMs for structural images

In [ ]:
# https://mriqc.readthedocs.io/en/latest/iqms/t1w.html

In [9]:
iqm_eids = []
iqm_generated = []

for each_eid in subjects_eid:
    each_eid = str(int(each_eid))
    try:
        with open(mriqc_generated + f"/sub-{each_eid}/ses-01/anat/sub-{each_eid}_ses-01_acq-ShortExample_run-01_T1w.json", 'r') as f_generated:
            json_file = json.load(f_generated)
            json_file['eid'] = each_eid
            iqm_generated.append(json_file)
            iqm_eids.append(each_eid)
    except Exception as e:
        print(f"{each_eid} not in IQM generated")
print(len(iqm_generated))
df_iqm_generated = pd.DataFrame.from_dict(iqm_generated)
df_iqm_generated

1741665 not in IQM generated
199


,bids_meta,cjv,cnr,efc,fber,fwhm_avg,fwhm_x,fwhm_y,fwhm_z,icvs_csf,...,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max,eid
0,"{'acq_id': 'ShortExample', 'dataset': '<unset>...",0.642051,0.683336,0.793480,-1.0,4.837930,5.01794,5.25028,4.24557,0.230904,...,979.878234,625995.386719,830.198266,1060.208636,69.276003,0.258228,0.507420,0.546802,0.853482,1062520
1,"{'acq_id': 'ShortExample', 'dataset': '<unset>...",0.673017,0.673899,0.768341,-1.0,5.061783,4.68525,5.20118,5.29892,0.228548,...,979.707929,604407.826759,851.088136,1063.158942,63.046625,0.269217,0.513184,0.556970,0.855232,2057779
2,"{'acq_id': 'ShortExample', 'dataset': '<unset>...",0.561088,0.759680,0.792867,-1.0,4.503493,4.70730,4.93346,3.86972,0.229340,...,985.394307,608079.936252,846.715612,1057.083021,62.972440,0.275491,0.535677,0.546274,0.871435,4450963
3,"{'acq_id': 'ShortExample', 'dataset': '<unset>...",0.654459,0.676685,0.785953,-1.0,4.762540,4.81818,5.14798,4.32146,0.235997,...,980.847554,621680.499209,839.365303,1061.222535,66.799494,0.260432,0.507391,0.543292,0.857299,1548100
4,"{'acq_id': 'ShortExample', 'dataset': '<unset>...",0.655078,0.693701,0.780867,-1.0,4.817877,5.06013,5.41609,3.97741,0.230530,...,968.397316,632961.973431,750.673579,1056.233188,95.940722,0.245139,0.499825,0.518288,0.861195,4380749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,"{'acq_id': 'ShortExample', 'dataset': '<unset>...",0.623340,0.744191,0.781399,-1.0,4.489077,4.67449,5.21532,3.57742,0.229327,...,976.880083,639512.850189,789.133840,1058.251523,82.460796,0.252035,0.496115,0.520764,0.862490,2473256
195,"{'acq_id': 'ShortExample', 'dataset': '<unset>...",0.643291,0.672521,0.790047,-1.0,5.308637,4.92179,5.49408,5.51004,0.228552,...,987.238423,623636.316548,881.458827,1061.369219,53.200483,0.275631,0.523150,0.550079,0.865195,5514758
196,"{'acq_id': 'ShortExample', 'dataset': '<unset>...",0.658319,0.654709,0.797515,-1.0,4.588120,4.63852,4.75270,4.37314,0.232659,...,970.343885,622193.590093,824.428813,1053.523362,69.238299,0.270395,0.524968,0.552642,0.860329,1937427
197,"{'acq_id': 'ShortExample', 'dataset': '<unset>...",0.574816,0.750382,0.783853,-1.0,4.990773,4.71636,5.09932,5.15664,0.228235,...,984.461470,600014.348262,855.964837,1056.264012,59.665346,0.267959,0.520581,0.539720,0.875135,1251663


In [10]:
iqm_original = []
for each_eid in iqm_eids:
    with open(mriqc_original + f"/sub-{each_eid}/ses-01/anat/sub-{each_eid}_ses-01_acq-ShortExample_run-01_T1w.json", 'r') as f_original:
        json_file = json.load(f_original)
        json_file['eid'] = each_eid
        iqm_original.append(json_file)
print(len(iqm_original))
df_iqm_original = pd.DataFrame.from_dict(iqm_original)
df_iqm_original

199


,bids_meta,cjv,cnr,efc,fber,fwhm_avg,fwhm_x,fwhm_y,fwhm_z,icvs_csf,...,summary_wm_median,summary_wm_n,summary_wm_p05,summary_wm_p95,summary_wm_stdv,tpm_overlap_csf,tpm_overlap_gm,tpm_overlap_wm,wm2max,eid
0,"{'acq_id': 'ShortExample', 'dataset': '<unset>...",0.563732,1.117477,0.857885,1.666768,5.345983,5.15648,5.26214,5.61933,0.237243,...,965.136894,598703.407509,763.616400,1053.720197,88.740299,0.250973,0.521178,0.559860,0.859727,1062520
1,"{'acq_id': 'ShortExample', 'dataset': '<unset>...",0.568678,1.232679,0.851715,1.479047,5.344720,4.87495,5.35822,5.80099,0.242443,...,963.996663,570545.326880,751.073817,1061.701519,94.718994,0.235072,0.514404,0.564929,0.854347,2057779
2,"{'acq_id': 'ShortExample', 'dataset': '<unset>...",0.501404,1.299201,0.857272,1.655845,4.991967,4.83592,4.91889,5.22109,0.243025,...,977.768613,576145.589808,804.752362,1057.192947,76.220781,0.265205,0.544065,0.551638,0.875295,4450963
3,"{'acq_id': 'ShortExample', 'dataset': '<unset>...",0.557741,1.143765,0.847297,1.430805,5.255213,4.94483,5.30581,5.51500,0.232049,...,962.307505,598645.612732,730.638540,1054.143505,99.571204,0.240832,0.506408,0.552401,0.858911,1548100
4,"{'acq_id': 'ShortExample', 'dataset': '<unset>...",0.588114,0.997600,0.847260,1.437306,5.391360,5.13421,5.37587,5.66400,0.225970,...,946.785801,611476.829411,652.564073,1048.266475,124.850297,0.232695,0.506730,0.534215,0.850398,4380749
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,"{'acq_id': 'ShortExample', 'dataset': '<unset>...",0.552880,1.093339,0.850196,1.709567,5.181780,4.85414,5.35066,5.34054,0.230655,...,969.700992,614367.285766,725.162866,1059.352196,103.346044,0.235344,0.503291,0.530757,0.853214,2473256
195,"{'acq_id': 'ShortExample', 'dataset': '<unset>...",0.537177,1.249952,0.847451,1.652102,5.538457,5.13391,5.68100,5.80046,0.243292,...,975.316793,590248.508791,799.583867,1059.523335,78.383597,0.244610,0.519408,0.552760,0.864228,5514758
196,"{'acq_id': 'ShortExample', 'dataset': '<unset>...",0.574929,1.152807,0.850499,1.580397,4.935047,4.82414,4.88623,5.09477,0.238719,...,949.130183,599549.108806,708.032398,1051.901780,106.736749,0.249168,0.522156,0.555997,0.841746,1937427
197,"{'acq_id': 'ShortExample', 'dataset': '<unset>...",0.509259,1.293212,0.852987,1.563537,5.181870,4.90140,5.22511,5.41910,0.245707,...,973.466415,573174.599126,777.452020,1058.355269,85.342059,0.250650,0.525814,0.543151,0.866959,1251663


In [ ]:
'''
iqm_differences = []


for index in range(len(iqm_original)):
    differences = {}
    for key, value in iqm_original[index].items():
        try:
            differences[key] = ((value - iqm_generated[index][key])/value)**2
        except:
            continue
    iqm_differences.append(differences)
print(len(iqm_differences))
'''

### Measures based on noise measurements

In [11]:
# coefficient of joint variation (CJV)
# yes
'''
The cjv of GM and WM was proposed as objective function by [Ganzetti2016] 
for the optimization of INU correction algorithms. Higher values are related 
to the presence of heavy head motion and large INU artifacts. Lower values are better.

# without post-processing (histogram matching )
The CJV was from $0.62 \pm 0.07$ to $0.76 \pm 0.10$, which indicates the increase of INU artifacts,
''' 

print(f"Original CJV : {df_iqm_original['cjv'].describe()}")
print(f"Generated CJV : {df_iqm_generated['cjv'].describe()}")

Original CJV : count    199.000000
mean       0.583029
std        0.064206
min        0.474459
25%        0.537002
50%        0.571573
75%        0.612836
max        0.870383
Name: cjv, dtype: float64
Generated CJV : count    199.000000
mean       0.664809
std        0.060111
min        0.541221
25%        0.623024
50%        0.659257
75%        0.694470
max        0.894399
Name: cjv, dtype: float64


In [12]:
# contrast-to-noise ratio
# yes
'''
The cnr [Magnota2006], is an extension of the SNR calculation to evaluate how separated 
the tissue distributions of GM and WM are. Higher values indicate better quality.


the CNR was from $1.09 \pm 0.12$ to $0.65 \pm 0.09$
''' 
print(f"Original CNR : {df_iqm_original['cnr'].describe()}")
print(f"Generated CNR : {df_iqm_generated['cnr'].describe()}")

Original CNR : count    199.000000
mean       1.131841
std        0.136128
min        0.710615
25%        1.040915
50%        1.131575
75%        1.225988
max        1.475575
Name: cnr, dtype: float64
Generated CNR : count    199.000000
mean       0.672279
std        0.061432
min        0.483284
25%        0.640146
50%        0.676685
75%        0.712991
max        0.808905
Name: cnr, dtype: float64


In [13]:
# signal-to-noise ratio.
# no 
'''
calculated within the tissue mask.
''' 
print(f"Original SNR : {df_iqm_original['snr_total'].describe()}")
print(f"Generated SNR : {df_iqm_generated['snr_total'].describe()}")

Original SNR : count    199.000000
mean       4.436020
std        0.663830
min        2.842207
25%        4.017684
50%        4.429105
75%        4.780709
max        6.424633
Name: snr_total, dtype: float64
Generated SNR : count    199.000000
mean       7.263185
std        1.029595
min        4.612870
25%        6.544949
50%        7.313169
75%        8.021943
max       10.124950
Name: snr_total, dtype: float64


In [14]:
# Dietrich’s SNR (SNRd)
# no
'''
as proposed by [Dietrich2007], using the air background as reference.
''' 
print(f"Original SNRd : {df_iqm_original['snrd_total'].describe()}")
print(f"Generated SNRd : {df_iqm_generated['snrd_total'].describe()}")

Original SNRd : count    199.000000
mean       1.459639
std        0.088701
min        1.118478
25%        1.398936
50%        1.457231
75%        1.526408
max        1.676679
Name: snrd_total, dtype: float64
Generated SNRd : count    199.000000
mean       1.616610
std        0.071637
min        1.315945
25%        1.577742
50%        1.623226
75%        1.660710
max        1.859085
Name: snrd_total, dtype: float64


In [15]:
# Mortamet’s quality index 2
# no
'''
Mortamet’s quality index 2 (QI2) is a calculation of the goodness-of-fit 
of a distribution on the air mask, once the artifactual intensities detected 
for computing the QI1 index have been removed [Mortamet2009]. Lower values are better.
'''
print(f"Original QI2 : {df_iqm_original['qi_2'].describe()}")
print(f"Generated QI2 : {df_iqm_generated['qi_2'].describe()}")

Original QI2 : count    199.000000
mean       0.002202
std        0.000286
min        0.001683
25%        0.002001
50%        0.002152
75%        0.002337
max        0.003146
Name: qi_2, dtype: float64
Generated QI2 : count    199.000000
mean       0.002223
std        0.000175
min        0.001761
25%        0.002113
50%        0.002224
75%        0.002299
max        0.002913
Name: qi_2, dtype: float64


### Measures based on information theory 

In [16]:
# The EFC
# Maybe - Measure with all others ***
'''
The EFC [Atkinson1997] uses the Shannon entropy of voxel intensities 
as an indication of ghosting and blurring induced by head motion. Lower values are better.

The original equation is normalized by the maximum entropy, so that the EFC can be 
compared across images with different dimensions.
'''
print(f"Original EFC : {df_iqm_original['efc'].describe()}")
print(f"Generated EFC : {df_iqm_generated['efc'].describe()}")

Original EFC : count    199.000000
mean       0.852028
std        0.006506
min        0.826507
25%        0.847969
50%        0.852347
75%        0.856448
max        0.866866
Name: efc, dtype: float64
Generated EFC : count    199.000000
mean       0.785042
std        0.009624
min        0.751713
25%        0.778886
50%        0.785859
75%        0.790525
max        0.822931
Name: efc, dtype: float64


In [17]:
# The FBER
# no
'''
The FBER [Shehzad2015], defined as the mean energy of image values within 
the head relative to outside the head [QAP-measures]. Higher values are better.
'''
print(f"Original FBER : {df_iqm_original['fber'].describe()}")
print(f"Generated FBER : {df_iqm_generated['fber'].describe()}")

Original FBER : count    199.000000
mean       1.500814
std        0.183470
min        1.081957
25%        1.389242
50%        1.499828
75%        1.610660
max        2.018771
Name: fber, dtype: float64
Generated FBER : count    199.0
mean      -1.0
std        0.0
min       -1.0
25%       -1.0
50%       -1.0
75%       -1.0
max       -1.0
Name: fber, dtype: float64


### Measures targeting specific artifacts

In [18]:
# summary statistics (Range and median) 
# maybe
'''
summary statistics (range and median) of the INU field (bias field) as extracted 
by the N4ITK algorithm [Tustison2010]. Values closer to 1.0 are better, 
values further from zero indicate greater RF field inhomogeneity.
'''
print(f"Original INU med: {df_iqm_original['inu_med'].describe()}")
print(f"Generated INU med: {df_iqm_generated['inu_med'].describe()}")
print()
print(f"Original INU Range: {df_iqm_original['inu_range'].describe()}")
print(f"Generated INU Range: {df_iqm_generated['inu_range'].describe()}")

Original INU med: count    199.000000
mean       0.400910
std        0.086044
min        0.249943
25%        0.336127
50%        0.383631
75%        0.436006
max        0.773502
Name: inu_med, dtype: float64
Generated INU med: count    199.000000
mean       0.181209
std        0.036003
min        0.115527
25%        0.157808
50%        0.173104
75%        0.196420
max        0.357518
Name: inu_med, dtype: float64

Original INU Range: count    199.000000
mean       0.046947
std        0.017879
min        0.027352
25%        0.037037
50%        0.042627
75%        0.051443
max        0.169675
Name: inu_range, dtype: float64
Generated INU Range: count    199.000000
mean       0.029530
std        0.009897
min        0.014015
25%        0.023356
50%        0.027821
75%        0.032581
max        0.095061
Name: inu_range, dtype: float64


In [19]:
# QI1
# no
'''
Detect artifacts in the image using the method described in [Mortamet2009]. The QI1 is the proportion 
of voxels with intensity corrupted by artifacts normalized by the number of voxels in the background. 
Lower values are better.
'''
print(f"Original QI1: {df_iqm_original['qi_1'].describe()}")
print(f"Generated QI1: {df_iqm_generated['qi_1'].describe()}")

Original QI1: count    199.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: qi_1, dtype: float64
Generated QI1: count    199.0
mean       0.0
std        0.0
min        0.0
25%        0.0
50%        0.0
75%        0.0
max        0.0
Name: qi_1, dtype: float64


In [20]:
# White-matter to maximum intensity ratio 
# no
'''
The white-matter to maximum intensity ratio is the median intensity within the WM 
mask over the 95% percentile of the full intensity distribution, that captures the existence 
of long tails due to hyper-intensity of the carotid vessels and fat. 
Values should be around the interval [0.6, 0.8].
'''
print(f"Original wm2max: {df_iqm_original['wm2max'].describe()}")
print(f"Generated wm2max: {df_iqm_generated['wm2max'].describe()}")

Original wm2max: count    199.000000
mean       0.848967
std        0.019265
min        0.758566
25%        0.841459
50%        0.853715
75%        0.861251
max        0.876408
Name: wm2max, dtype: float64
Generated wm2max: count    199.000000
mean       0.856912
std        0.014513
min        0.797016
25%        0.851225
50%        0.859162
75%        0.866381
max        0.881191
Name: wm2max, dtype: float64


### Other measures

In [21]:
# The FWHM 
# maybe - measure with other images
'''
The FWHM of the spatial distribution of the image intensity values in units of voxels [Forman1995]. 
Lower values are better, higher values indicate a blurrier image. 
Uses the gaussian width estimator filter implemented in AFNI’s 3dFWHMx:
'''
print(f"Original FWHM: {df_iqm_original['fwhm_avg'].describe()}")
print(f"Generated FWHM: {df_iqm_generated['fwhm_avg'].describe()}")

Original FWHM: count    199.000000
mean       5.207189
std        0.184685
min        4.594257
25%        5.073740
50%        5.196287
75%        5.341718
max        5.737333
Name: fwhm_avg, dtype: float64
Generated FWHM: count    199.000000
mean       4.749533
std        0.317852
min        3.884887
25%        4.520078
50%        4.804737
75%        4.962093
max        5.514110
Name: fwhm_avg, dtype: float64


In [22]:
# Volume Fraction
# no
"""
the ICV fractions of CSF, GM and WM. They should move within a normative range.
"""
print(f"Original ICV of Gray Matter: {df_iqm_original['icvs_gm'].describe()}")
print(f"Generated ICV of Gray Matter: {df_iqm_generated['icvs_gm'].describe()}")

Original ICV of Gray Matter: count    199.000000
mean       0.385056
std        0.004187
min        0.366039
25%        0.382936
50%        0.385331
75%        0.387810
max        0.396001
Name: icvs_gm, dtype: float64
Generated ICV of Gray Matter: count    199.000000
mean       0.377602
std        0.006939
min        0.361586
25%        0.372806
50%        0.376819
75%        0.382736
max        0.395176
Name: icvs_gm, dtype: float64


In [23]:
# the rPVe of CSF, GM and WM. Lower values are better
# no
print(f"Original rPVe of Gray Matter: {df_iqm_original['rpve_gm'].describe()}")
print(f"Generated rPVe of Gray Matter: {df_iqm_generated['rpve_gm'].describe()}")

Original rPVe of Gray Matter: count    199.000000
mean       1.246770
std        0.032705
min        1.128145
25%        1.227133
50%        1.250914
75%        1.266480
max        1.354822
Name: rpve_gm, dtype: float64
Generated rPVe of Gray Matter: count    199.000000
mean       1.209142
std        0.032122
min        1.122291
25%        1.189539
50%        1.211057
75%        1.228165
max        1.331089
Name: rpve_gm, dtype: float64


In [24]:
# Summary stats
# no
'''
Mean, median, median absolute deviation (mad), standard deviation, 
kurtosis, 5% percentile, 95% percentile and number of voxels of the 
distribution of background, CSF, GM and WM.
'''
print(f"Original median absolute deviation of Gray Matter: {df_iqm_original['summary_gm_mad'].describe()}")
print(f"Generated median absolute deviation of Gray Matter: {df_iqm_generated['summary_gm_mad'].describe()}")

Original median absolute deviation of Gray Matter: count    199.000000
mean     138.691810
std       10.992449
min      113.380441
25%      131.343875
50%      138.804996
75%      145.776250
max      169.621276
Name: summary_gm_mad, dtype: float64
Generated median absolute deviation of Gray Matter: count    199.000000
mean      92.299591
std        6.277038
min       76.944656
25%       88.085688
50%       92.320141
75%       96.275510
max      116.267730
Name: summary_gm_mad, dtype: float64


In [25]:
# The overlap of the TPMs
# maybe
'''
The overlap of the TPMs estimated from the image and the corresponding maps 
from the ICBM nonlinear-asymmetric 2009c template. Higher values are better.
'''
print(f"Original Gray Matter Overlap: {df_iqm_original['tpm_overlap_gm'].describe()}")
print(f"Generated Gray Matter Overlap: {df_iqm_generated['tpm_overlap_gm'].describe()}")
print()
print(f"Original CSF Overlap: {df_iqm_original['tpm_overlap_csf'].describe()}")
print(f"Generated CSF Overlap: {df_iqm_generated['tpm_overlap_csf'].describe()}")
print()
print(f"Original White Matter Overlap: {df_iqm_original['tpm_overlap_wm'].describe()}")
print(f"Generated White Matter Overlap: {df_iqm_generated['tpm_overlap_wm'].describe()}")


Original Gray Matter Overlap: count    199.000000
mean       0.515092
std        0.013510
min        0.467328
25%        0.506054
50%        0.516585
75%        0.525189
max        0.545562
Name: tpm_overlap_gm, dtype: float64
Generated Gray Matter Overlap: count    199.000000
mean       0.511335
std        0.013196
min        0.465767
25%        0.503660
50%        0.511781
75%        0.520344
max        0.543860
Name: tpm_overlap_gm, dtype: float64

Original CSF Overlap: count    199.000000
mean       0.241396
std        0.011853
min        0.200226
25%        0.234457
50%        0.242845
75%        0.249707
max        0.265205
Name: tpm_overlap_csf, dtype: float64
Generated CSF Overlap: count    199.000000
mean       0.262056
std        0.010745
min        0.228508
25%        0.254699
50%        0.262371
75%        0.268798
max        0.288789
Name: tpm_overlap_csf, dtype: float64

Original White Matter Overlap: count    199.000000
mean       0.550453
std        0.013509
min        

In [ ]:
differences = pd.DataFrame.from_dict(differences, orient='index')

In [ ]:
differences = differences[~differences.index.str.contains('_gm')]
differences = differences[~differences.index.str.contains('_wm')]
differences = differences[~differences.index.str.contains('_csf')]

In [ ]:
differences.sort_values(by=[0], ascending=False)